# Import dependencies

In [61]:
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import time
import numpy as np

# Download dataset

In [3]:
training_data = KMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = KMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████████████████████| 18165135/18165135 [00:06<00:00, 2840321.80it/s]


Extracting data/KMNIST/raw/train-images-idx3-ubyte.gz to data/KMNIST/raw



100%|█████████████████████████████████| 29497/29497 [00:00<00:00, 467430.05it/s]


Extracting data/KMNIST/raw/train-labels-idx1-ubyte.gz to data/KMNIST/raw



100%|████████████████████████████| 3041136/3041136 [00:01<00:00, 2125047.50it/s]


Extracting data/KMNIST/raw/t10k-images-idx3-ubyte.gz to data/KMNIST/raw



100%|██████████████████████████████████| 5120/5120 [00:00<00:00, 7710892.81it/s]

Extracting data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to data/KMNIST/raw



# Implement Lenet

In [36]:
class Lenet(nn.Module):
    def __init__(self, in_channels=1, num_cls=10):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=20,
            kernel_size=(5, 5)
        )
        self.conv2 = nn.Conv2d(
            in_channels=20,
            out_channels=50,
            kernel_size=(5, 5)
        )
        self.pool = nn.MaxPool2d(
            kernel_size=(2, 2),
            stride=(2, 2)
        )
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=800, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=num_cls)
        self.logSoftMax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.logSoftMax(x)
        return x

# Training

## Hyperparameters

In [17]:
INIT_LR = 0.001
EPOCHS = 3
BATCH_SIZE = 64

TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.25

## Data loader

In [18]:
numTrainSamples = int(TRAIN_SPLIT*len(training_data))
numValSamples = int(VAL_SPLIT*len(training_data))

training_data, val_data = random_split(
    training_data,
    [numTrainSamples, numValSamples],
    generator=torch.Generator().manual_seed(42)
)

In [19]:
train_dataloader = DataLoader(
    dataset=training_data, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_dataloader = DataLoader(
    dataset=val_data, 
    batch_size=BATCH_SIZE,
)
test_dataloader = DataLoader(
    dataset=test_data, 
    batch_size=BATCH_SIZE,
)

## Model, optimizer and loss function

In [52]:
model = Lenet(
    in_channels=1,
    num_cls=len(training_data.dataset.classes)
)
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=INIT_LR
)
loss_fn = nn.NLLLoss()

In [53]:
H = {
    "train_loss": [],
    "train_accuracy": [],
    "val_loss": [],
    "val_accuracy": []
}
trainSteps = len(train_dataloader) / BATCH_SIZE
valSteps = len(val_dataloader) / BATCH_SIZE
matplotlib.use("Agg")
print('[INFO] Training the network...')

[INFO] Training the network...


## Start training

In [54]:
startTime = time.time()
model.train()
for t in range(EPOCHS):
    totalTrainLoss = 0
    totalValLoss = 0

    trainCorrect = 0
    valCorrect = 0
    
    for (inputs, labels) in train_dataloader:
        # forward
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
    
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        totalTrainLoss += loss
        trainCorrect += (outputs.argmax(1) == labels).type(torch.float).sum()

    with torch.no_grad():
        for (inputs, labels) in val_dataloader:
            # forward
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            totalValLoss += loss
            valCorrect += (outputs.argmax(1) == labels).type(torch.float).sum() 

    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    trainCorrect /= len(train_dataloader.dataset)
    valCorrect /= len(val_dataloader.dataset)
    
    H["train_loss"].append(avgTrainLoss.detach().numpy())
    H["train_accuracy"].append(trainCorrect.detach().numpy())
    H["val_loss"].append(avgValLoss.detach().numpy()) 
    H["val_accuracy"].append(valCorrect.detach().numpy()) 

    print(f"[INFO] EPOCH: {t + 1}/{EPOCHS}") 
    print(f"Train loss: {avgTrainLoss:.6f}, train accuracy: {trainCorrect:.4f}")
    print(f"Val loss: {avgValLoss:.6f}, val accuracy: {valCorrect:.4f}")

endTime = time.time()

[INFO] EPOCH: 1/3
Train loss: 22.309212, train accuracy: 0.8906
Val loss: 8.939799, val accuracy: 0.9592
[INFO] EPOCH: 2/3
Train loss: 6.136609, train accuracy: 0.9711
Val loss: 5.455637, val accuracy: 0.9747
[INFO] EPOCH: 3/3
Train loss: 3.529879, train accuracy: 0.9832
Val loss: 4.991375, val accuracy: 0.9778


In [62]:
print(f"[INFO] Time taken to train model: {endTime - startTime:2f}")

print("[INFO] Evaluating network: ...")

with torch.no_grad():
    model.eval()
    preds = []

    for (inputs, labels) in test_dataloader:
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        preds.extend(outputs.argmax(1).cpu().numpy())

print(
    classification_report(
        test_data.targets.cpu().numpy(), 
        np.array(preds), 
        target_names=test_data.classes
    )
)

[INFO] Time taken to train model: 61.949633
[INFO] Evaluating network: ...
              precision    recall  f1-score   support

           o       0.91      0.98      0.94      1000
          ki       0.91      0.91      0.91      1000
          su       0.94      0.89      0.91      1000
         tsu       0.91      0.97      0.94      1000
          na       0.93      0.92      0.92      1000
          ha       0.97      0.92      0.95      1000
          ma       0.86      0.98      0.92      1000
          ya       0.98      0.88      0.93      1000
          re       0.96      0.91      0.93      1000
          wo       0.97      0.95      0.96      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [64]:
plt.style.use('ggplot')
plt.figure()
plt.plot(H['train_loss'], label='train_loss')
plt.plot(H['val_loss'], label='val_loss')
plt.title('Training loss and accuracy on dataset')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='lower left')
plt.savefig('./figures/Loss.png')

plt.figure()
plt.plot(H['train_accuracy'], label='train_accuracy')
plt.plot(H['val_accuracy'], label='val_accuracy')
plt.title('Training loss and accuracy on dataset')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower left')
plt.savefig('./figures/Accuracy.png')

torch.save(model, './model/test_model')